In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='subhalo.yml'
displayParameters=True

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

# Relation expected for isothermal sphere for LCDM cosmology used in Sussing
Delta=800
# Relation expected for isothermal sphere for LCDM cosmology used in Millennium
Delta=200

haloFile=parameters['inputFiles']['haloFile']
subhaloFile=parameters['inputFiles']['subhaloFile']
save_prefix=parameters['save_prefix']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']
baryonFraction=parameters['cosmo']['baryonFraction']

In [ ]:
# Read in halo table
with h5py.File(haloFile,'r') as f:
    # Get the data from the snapshot
    halos=f['Halos'][:]
    
# Read in subhalo table
with h5py.File(subhaloFile,'r') as f:    
    #Get the data from the snapshot
    subs=f['Halos'][:]

In [ ]:
# Filter out the halos and subhalos in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
halos=halos[(graphMin<=halos['graph_ID']) & (halos['graph_ID']<=graphMax) & (snapMin<=halos['snap_ID']) & (halos['snap_ID']<=snapMax)]
subs=subs[(graphMin<=subs['graph_ID']) & (subs['graph_ID']<=graphMax) & (snapMin<=subs['snap_ID']) & (subs['snap_ID']<=snapMax)]

In [ ]:
# Match up galaxies with subhalos and/or halos
# Note that orphan galaxies can have no halo if there is no descendant for the host halo in the previous snap.
# Note that orphan galaxies have no subhalo.
n_sub=len(subs)
halo_loc=np.zeros(n_sub,dtype=np.int32)
for i_sub in range(n_sub):
    halo_loc[i_sub]=np.where(np.logical_and(halos['graph_ID']==subs['graph_ID'][i_sub],halos['halo_gid']==subs['halo_gid'][i_sub]))[0][0]
halos=halos[halo_loc]

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='velocityDispersion':
            xdata=subs['mass']
            ydata=subs['half_mass_virial_speed']/np.sqrt(2.)
            ydata2=subs['rms_speed']/np.sqrt(3.)
            from astropy.cosmology import FlatLambdaCDM
            cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
            ydata3=((10*c.M_sun*c.G*cosmo.H(0))**(1/3)).to(u.km/u.s)*(Delta/200)**(1/6)*xdata**(1/3)/np.sqrt(2.)
            plt.plot(xdata,ydata3,'-',label='SIS model, $\Delta='+str(Delta)+'$')
            plt.plot(xdata,ydata2,plotProps['markerType'],label=r'From rms speed')
        elif plotName=='virialTemperature':
            xdata=subs['mass']
            ydata=subs['temperature']
            ydata2=((u.km/u.s)**2*1e-27*u.kg/c.k_B).to(u.K)*subs['rms_speed']**2/3.
            from astropy.cosmology import FlatLambdaCDM
            cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
            ydata3=((10*c.M_sun*c.G*cosmo.H(0))**(2/3)*1e-27*u.kg/c.k_B).to(u.K)*(Delta/200)**(1/3)*0.5*xdata**(2/3)
            plt.plot(xdata,ydata3,'-',label='SIS model, $\Delta='+str(Delta)+'$')
            plt.plot(xdata,ydata2,plotProps['markerType'],label=r'From rms speed')
        elif plotName=='virialSpeedVersusHalo':
            xdata=halos['half_mass_virial_speed']
            ydata=subs['half_mass_virial_speed']
            plt.plot(xdata,xdata,'k:')
        elif plotName=='baryonFraction':
            xdata=subs['mass']
            ydata=subs['mass_baryon']
            plt.plot(xdata,0.*xdata+baryonFraction,'-')
        elif plotName=='hotGas':
            xdata=subs['mass']
            ydata=subs['mass_gas_hot']
            plt.plot(xdata,xdata*baryonFraction,'-')
        elif plotName=='hotGasMetallicity':
            xdata=subs['mass']
            ydata=subs['mass_metals_gas_hot']/subs['mass_gas_hot']
        elif plotName=='stellarHaloMassRatio':
            xdata=subs['mass']
            ydata=subs['mass_stars']/subs['mass']
            plt.ylim([0.,1.1*np.max(ydata)])
        elif plotName=='starFormationRate':
            xdata=subs['mass']
            ydata=subs['star_formation_rate']
        else:
            print('Plot type',plotName,'not yet implemented')
            continue # with next plot request
        plt.plot(xdata,ydata,plotProps['markerType'],label=plotProps['label'])
        if plotProps['legend']: plt.legend()
        if plotProps['save_png']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.png',bbox_inches='tight')
        if plotProps['save_pdf']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.pdf',bbox_inches='tight')